Для того, чтобы выбрать цену для каждого товара, нам необходимо брать во внимание бизнес-план (по выручке и проценте прибыли) по каждой категории, включённой в наше ML ценообразование.

Пусть мы имеем 5 матриц: Costs, GMV, Margin, Prices, Qty

💡 GMV = qty * price = (шт. продано) * (цена) 

💡 Маржинальность = (price – cost) / price = (цена – себестоимость) / цена

На вход нашего сервиса поступают:

1) DataFrame из 4 колонок:

• sku – ID товара (SKU = Stock Keeping Unit)

• cost – себестоимость товара (цена закупки)

• price – цена-кандидат, среди которых нужно выбрать 1 на каждый товар

• qty – прогноз, сколько шт. будет продано в следующие 7 дней
________________________________________
________________________________________
2) Целевая маржинальность:

💡 Целевая маржинальность – уровень взвешенной маржи, ниже которого мы не хотим опускаться.

💡 Взвешенная маржинальность – средняя маржинальность категории, взвешенная на GMV.


На выходе сервиса ожидается:

Маска для описанного выше DataFrame в формате [False, False, True, ..., False, True], применив которую к нему мы оставим выбранные цены.

#### Задача:

Разработать алгоритм, рекомендующий цену для каждого товара, который максимизирует выручку при условии, что взвешенная маржа больше таргет-маржи.

Ответ принимается в формате класса PriceRecommender содержащий следующие методы:

• gmv(df: pd.DataFrame) -> pd.Series (превращает исходный датафрейм в pd.Series с GMV).

• margin(df: pd.DataFrame) -> pd.Series (превращает исходный датафрейм в pd.Series с маржой)

• weighted_margin(df: pd.DataFrame) -> float (превращает исходный датафрейм в средневзвешенную маржу)

• recommend_price(self, df: pd.DataFrame, target_margin: float) -> List[bool] (описанный выше сервис)

In [1]:
from typing import List
from dataclasses import dataclass
from copy import copy
import pandas as pd


@dataclass
class PriceRecommender:
    
    def recommend_price(self, df: pd.DataFrame, target: float) -> List[bool]:
        skus = df['sku'].unique()
        prices = {}
        
        for sku in skus:
            prices[sku] = df[df['sku']==sku]['price'].unique()
            
        revenue = PriceRecommender().gmv(df).sum()
        rec_price = {sku:prices[sku][0] for sku in prices}
        df_ = copy(df)
        
        for sku in prices:
            for price in prices[sku]:
                qty = df[(df['sku']==sku)&(df['price']==price)]['qty'].values[0]
                df_.loc[df['sku']==sku, 'price'] = price
                df_.loc[df['sku']==sku, 'qty'] = qty
                new_revenue = PriceRecommender().revenue(df_)
                if PriceRecommender().weighted_margin(df_) > target and new_revenue > revenue:
                    revenue = new_revenue            
                    rec_price[sku]=price
                
        mask = []
        for index, row in df.iterrows():
            mask.append(rec_price[row['sku']]==row['price'])                   
        return mask

    @staticmethod
    def gmv(df: pd.DataFrame) -> pd.Series:
        df_ = copy(df)
        df_['gmv'] = df_['qty']*df_['price']
        return df_['gmv']

    @staticmethod
    def margin(df: pd.DataFrame) -> pd.Series:
        df_ = copy(df)
        df_['margin'] = (df_['price']-df_['cost'])/df_['price']
        return df_['margin']

    @staticmethod
    def weighted_margin(df: pd.DataFrame) -> float:
        df_ = copy(df)
        df_['gmv'] = PriceRecommender().gmv(df_)
        df_['margin'] = PriceRecommender().margin(df_)
        revenue = df_['gmv'].sum()
        agg_df = df_.groupby(['sku', 'price'], as_index=False).agg({'gmv':'sum', 'margin':'mean'})
        agg_df['pennetration'] = agg_df['gmv']/revenue
        weighted_margin = (agg_df['margin']*agg_df['pennetration']).sum()
        return weighted_margin
    
    @staticmethod
    def revenue(df: pd.DataFrame) -> float:
        return sum(df['qty']*df['price'])


In [2]:
rows = [
    [1, 40, 40, 100],
    [1, 40, 45, 80],
    [1, 40, 50, 55],
    [2, 100, 200, 300],
    [2, 100, 220, 280],
    [2, 100, 240, 200],
]

df = pd.DataFrame(rows, columns=["sku", "cost", "price", "qty"])
df

,sku,cost,price,qty
0,1,40,40,100
1,1,40,45,80
2,1,40,50,55
3,2,100,200,300
4,2,100,220,280
5,2,100,240,200


In [3]:
print('GMV:')
print(PriceRecommender().gmv(df))   
print()
print('Margin:')
print(PriceRecommender().margin(df))
print()
print('Weighted margin:') 
print(round(PriceRecommender().weighted_margin(df), 2))
print()
print('Recommend price:') 
print(PriceRecommender().recommend_price(df, 0.2))

GMV:
0     4000
1     3600
2     2750
3    60000
4    61600
5    48000
Name: gmv, dtype: int64

Margin:
0    0.000000
1    0.111111
2    0.200000
3    0.500000
4    0.545455
5    0.583333
Name: margin, dtype: float64

Weighted margin:
0.51

Recommend price:
[True, False, False, False, True, False]
